In [39]:
from behavioural_benchmark.indicators import MemoisedIndicators
import numpy as np
from math import floor
import os
import random

To test and illustrate the indicators, let's construct examples where we know the answers. For each indicator we will create some data where we decide on the values of the indicators, and at the end we will assert that the calculated indicators match up with the expected ones.

## Regression indicators

For the regression based indicators, we can define graphs of two lines with set gradients. We can sample points along these lines, and save these to a file. Then later we can assert whether the regression indicators confirm the gradients we selected.

In [40]:
c = 100
x = np.array([j for j in range(0,100,2)])

def sample_points(m_1, m_2):
    y_1 = m_1 * x + c
    c_2 = y_1[floor(0.70 * len(y_1[y_1 >= 0]))]
    x_1 = x[y_1 >= c_2]
    y_1 = y_1[y_1 >= c_2]

    y_2 = m_2 * (x_1 + 1) + c_2
    x_2 = x_1 + max(x_1) + 1
    x_2 = x_2[y_2 >= 0]
    y_2 = y_2[y_2 >= 0]
    
    return np.concat([x_1, x_2]), np.concat([y_1, y_2])

def save_to_file(xf, yf, filename):
    data = np.column_stack((xf, yf))
    np.savetxt(f"example_data/{filename}.csv", data, delimiter=",", header=f"iteration,{filename}", comments="", fmt="%.4f")

# diversity
DRoC = -2
CRoC = -0.9
diversity_x, diversity_y = sample_points(DRoC, CRoC)
save_to_file(diversity_x, diversity_y, "diversity")

# fitness
ARoC_B = -1.8
LRoC_B = -1.0
fitness_x, fitness_y = sample_points(ARoC_B, LRoC_B)
save_to_file(fitness_x, fitness_y, "fitness")

# distance
ARoC_A = -2.05
LRoC_A = -0.75
distance_x, distance_y = sample_points(ARoC_A, LRoC_A)
save_to_file(distance_x, distance_y, "distance")

For the network based indicators, we need to construct interaction data, and trajectory data.

## Trajectory indicators

For the trajectory data, we can decide on a number of nodes and shared nodes. We can then simulate trajectories between these nodes, making sure that every node is visited at least once. We can create two runs, with shared nodes appearing in both runs. We can arbitrarily set the fitness of each node equal to 0, as it's not required for the indicators we are testing.

In [41]:
# STN
shared_nodes = list(range(20, 30))
nodes = {
    0: list(range(20)) + shared_nodes,
    1: shared_nodes + list(range(30, 30+20))
}
ntotal = len(np.unique(nodes[0] + nodes[1]))
nshared = len(shared_nodes)

file  = "example_data/stn.csv"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write("Run,Fitness1,Solution1,Fitness2,Solution2\n")
    for run in [0,1]:
        current_position = dict(zip(range(10), random.sample(nodes[run], 10)))
        
        # Create a set of nodes to ensure each one is selected at least once
        required_nodes = set(nodes[run])
        selected_nodes = set(current_position.values())  # Keep track of selected nodes
        for _step in range(10):
            for individual in range(10):
                # Ensure each node is selected at least once
                if len(selected_nodes) < len(required_nodes):
                    # Pick a node from the unselected ones
                    next_position = random.choice(list(required_nodes - selected_nodes))
                    selected_nodes.add(next_position)
                else:
                    # Once all nodes have been selected at least once, continue with random choice
                    next_position = random.choice(nodes[run])
                f.write(f"{run},0,{current_position[individual]},0,{next_position}\n")
                current_position[individual] = next_position

## Interaction  indicators

For the interaction data, we can create a small network of 5 individuals, resulting in a 5 x 5 grid. We can choose an IDRoC, and then model the interactions to result in the chosen IDRoC. We can also make sure the solution node has a weight equal to some chosen ISS.

In [42]:
# IN
IDRoC = 4
solution_index = 3
ISS = 30 / (2 * 10)

# let number subgraphs be of 1, 2, 3, 5 (y)
# with maximum edge_weight removed equal to 0.0, 0.25, 0.5, 1.0 (x)
# (5 - 1) / (1.0 - 0.0) = 4

arr = np.array([
    # 0   1   2   3   4
    [ 0, 20,  0,  0,  0], # 0
    [20,  0,  5,  0,  0], # 1
    [ 0,  5,  0, 10,  0], # 2
    [ 0,  0, 10,  0, 20], # 3
    [ 0,  0,  0, 20,  0]  # 4
])

zeroes = np.zeros(shape=arr.shape)

file  = "example_data/interaction.txt"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write(f"ig:#0 {' '.join(map(str, zeroes.flatten()))}\n")
    f.write(f"ig:#1 {' '.join(map(str, arr.flatten()))}\n")

## ENES & INFEASIBLE%

Finally, only the "odd one out" indicators are left. These are ENES, and infeasible percent. These are simple enough to verify.

In [43]:
total_iterations = 10
fitness_evaluations = total_iterations * 5 
infeasible_iterations = 2 * 5
global_best_fitness = 0
ENES = fitness_evaluations / total_iterations
INFEASIBLE_Percent = infeasible_iterations / total_iterations

file  = "example_data/metadata.json"
if os.path.exists(file):
    os.remove(file)
with open(file, "w") as f:
    f.write("{" + f'"total_iterations": {total_iterations}, "fitness_evaluations": {fitness_evaluations}, "infeasible_iterations": {infeasible_iterations}, "global_best_fitness": {global_best_fitness}, "solution_index": {solution_index}' + "}")

## Tests

In [44]:
k = MemoisedIndicators(path="example_data/")

def close_enough(expected, real):
    assert expected == round(real,4)

In [45]:
close_enough(DRoC, k.get_DRoC())
close_enough(CRoC, k.get_CRoC())
close_enough(ARoC_A, k.get_ARoC_A())
close_enough(LRoC_A, k.get_LRoC_A())
close_enough(ARoC_B, k.get_ARoC_B())
close_enough(LRoC_B, k.get_LRoC_B())

In [46]:
close_enough(ntotal, k.get_ntotal())
close_enough(nshared, k.get_nshared())

In [48]:
close_enough(IDRoC, k.get_IDRoC())
close_enough(ISS, k.get_ISS())

In [49]:
close_enough(ENES, k.get_ENES())
close_enough(INFEASIBLE_Percent, k.get_INFEASIBLE_Percent())